In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
rec = pd.read_csv('/kaggle/input/week-3/nfl pass catchers - week 3.csv', header=0)
rec.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/week-2-passing/pass catchers through week 2 - Sheet1.csv
/kaggle/input/week-3/nfl pass catchers - week 3.csv
/kaggle/input/passers-week-3/nfl passers 2024 - Accuracy.csv


,Rk,Player,Tm,Age,Pos,G,GS,Tgt\n▼,Rec,Yds,...,YBC/R,YAC,YAC/R,ADOT,BrkTkl,Rec/Br,Drop,Drop%,Int,Rat
0,1,Malik Nabers,NYG,21,WR,3,2,37,23,271,...,6.3,127,5.5,8.5,2,11.5,3,8.1,0,111.4
1,2,Amon-Ra St. Brown,DET,25,WR,3,3,32,21,207,...,7.4,52,2.5,8.2,1,21.0,2,6.3,1,81.1
2,3,Rashee Rice,KAN,24,WR,3,3,29,24,288,...,4.3,186,7.8,5.2,1,24.0,0,0.0,1,116.7
3,4,Amari Cooper,CLE,30,WR,3,3,29,12,113,...,7.9,18,1.5,12.8,0,NaN,6,20.7,0,75.8
4,5,DeVonta Smith,PHI,26,WR,3,3,28,21,239,...,7.7,77,3.7,10.8,0,NaN,0,0.0,3,72.5


In [2]:
print(rec.columns)

rec = rec.rename(columns={'Tgt\n▼': 'Tgt'})


Index(['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt\n▼', 'Rec', 'Yds',
       'TD', '1D', 'YBC', 'YBC/R', 'YAC', 'YAC/R', 'ADOT', 'BrkTkl', 'Rec/Br',
       'Drop', 'Drop%', 'Int', 'Rat'],
      dtype='object')


**Column Meanings:**
Age -- Player's age on December 31st of that year  
Pos -- Position  
G -- Games played  
GS -- Games started as an offensive or defensive player  
Tgt -- Pass Targets (since 1992, derived from NFL play-by-play data)  
Rec -- Receptions  
Yds -- Receiving Yards  
TD -- Receiving Touchdowns  
1D -- First downs receiving  
YBC -- Total yards passes traveled in the air before being caught  
YBC/R -- Yards before catch per reception  
YAC -- Yards after catch  
YAC/R -- Yards after catch per reception  
ADOT -- Average depth of target when targeted, whether completed or not. 25 targets/16 game pace required for leaderboards.  
BrkTkl -- Broken tackles on receptions  
Rec/Br -- Receptions per broken tackle  
Drop -- Dropped passes  
Drop% -- Dropped passes per target  
Int -- Interceptions on passes where targeted  
Rat -- Passer rating on passes when targeted. 25 targets/16 game pace required for leaderboards.  

In [3]:
# Group by position
grouped_rec = rec.groupby('Pos')
rb_rec = grouped_rec.get_group('RB')
wr_rec = grouped_rec.get_group('WR')
te_rec = grouped_rec.get_group('TE')
rb_rec.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Yds,...,YBC/R,YAC,YAC/R,ADOT,BrkTkl,Rec/Br,Drop,Drop%,Int,Rat
31,32,De'Von Achane,MIA,23,RB,3,2,19,17,173,...,-0.6,183,10.8,-0.6,1,17.0,0,0.0,0,122.1
32,33,Breece Hall,NYJ,23,RB,3,3,19,16,120,...,1.2,101,6.3,1.3,1,16.0,1,5.3,0,110.5
64,65,Aaron Jones,MIN,30,RB,3,3,14,12,97,...,-1.8,118,9.8,-1.1,0,NaN,0,0.0,0,119.3
66,67,Tony Pollard,TEN,27,RB,3,3,14,11,67,...,1.8,47,4.3,0.6,0,NaN,1,7.1,0,86.6
75,76,Rachaad White,TAM,25,RB,3,3,13,12,98,...,-3.8,143,11.9,-3.9,2,6.0,0,0.0,0,98.1


In [4]:
wr_rec.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Yds,...,YBC/R,YAC,YAC/R,ADOT,BrkTkl,Rec/Br,Drop,Drop%,Int,Rat
0,1,Malik Nabers,NYG,21,WR,3,2,37,23,271,...,6.3,127,5.5,8.5,2,11.5,3,8.1,0,111.4
1,2,Amon-Ra St. Brown,DET,25,WR,3,3,32,21,207,...,7.4,52,2.5,8.2,1,21.0,2,6.3,1,81.1
2,3,Rashee Rice,KAN,24,WR,3,3,29,24,288,...,4.3,186,7.8,5.2,1,24.0,0,0.0,1,116.7
3,4,Amari Cooper,CLE,30,WR,3,3,29,12,113,...,7.9,18,1.5,12.8,0,NaN,6,20.7,0,75.8
4,5,DeVonta Smith,PHI,26,WR,3,3,28,21,239,...,7.7,77,3.7,10.8,0,NaN,0,0.0,3,72.5


In [5]:
te_rec.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Yds,...,YBC/R,YAC,YAC/R,ADOT,BrkTkl,Rec/Br,Drop,Drop%,Int,Rat
21,22,Brock Bowers,LVR,22,TE,3,2,21,18,197,...,6.4,81,4.5,5.6,0,NaN,0,0.0,0,105.8
25,26,Trey McBride,ARI,25,TE,3,3,21,14,122,...,5.6,44,3.1,8.8,1,14.0,1,4.8,0,81.8
27,28,Dallas Goedert,PHI,29,TE,3,3,20,17,239,...,4.8,158,9.3,5.2,0,NaN,0,0.0,0,116.5
36,37,Mike Gesicki,CIN,29,TE,3,1,18,14,156,...,5.9,74,5.3,7.8,0,NaN,1,5.6,0,102.8
38,39,Hunter Henry,NWE,30,TE,3,3,18,12,136,...,5.6,69,5.8,5.6,0,NaN,1,5.6,0,89.1


In [6]:
# List of columns to normalize
cols = [
    'Player', 'Tgt', 'Rec', 'Yds', 'TD', '1D', 'YBC',
    'YAC', 'ADOT', 'BrkTkl', 'Drop%', 'Drop',
    'Int', 'Rat'
]

weights_wr = {
    'Tgt': 0.2,
    'Rec': 0.2,
    'Yds': 0.1,
    'TD': 0.3,
    '1D': 0.05,
    'YBC': 0.05,
    'YAC': 0.1,
    'ADOT': 0.05,
    'BrkTkl': 0.05,
    'Drop%': -0.25,
    'Int': -0.05,
    'Rat': 0.05
}

# Calculate weighted score
wr_rec['Score'] = sum(wr_rec[col] * weight for col, weight in weights_wr.items())

# Sort by score in descending order
ranked_wr = wr_rec.sort_values(by='Score', ascending=False)

# Get top 10 WR
top_10_wr = ranked_wr.head(10)
top_10_wr[cols].head()

/tmp/ipykernel_18/1695051736.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wr_rec['Score'] = sum(wr_rec[col] * weight for col, weight in weights_wr.items())


,Player,Tgt,Rec,Yds,TD,1D,YBC,YAC,ADOT,BrkTkl,Drop%,Drop,Int,Rat
6,Nico Collins,28,18,338,1,16.0,231,107,12.1,1,3.6,1,1,103.0
2,Rashee Rice,29,24,288,2,15.0,102,186,5.2,1,0.0,0,1,116.7
0,Malik Nabers,37,23,271,3,14.0,144,127,8.5,2,8.1,3,0,111.4
12,Chris Godwin,25,21,253,3,17.0,115,138,6.7,0,4.0,1,0,148.4
24,Justin Jefferson,21,14,273,3,10.0,181,92,12.2,0,0.0,0,0,149.3


In [7]:
# List of columns to normalize
cols = [
    'Player', 'Tgt', 'Rec', 'Yds', 'TD', '1D', 'YBC',
    'YAC', 'ADOT', 'BrkTkl', 'Drop%', 'Drop',
    'Int', 'Rat'
]


weights_te = {
    'Tgt': 0.15,
    'Rec': 0.2,
    'Yds': 0.15,
    'TD': 0.2,
    '1D': 0.1,
    'YBC': 0.05,
    'YAC': 0.1,
    'ADOT': 0.05,
    'BrkTkl': 0.1,
    'Drop%': -0.25,
    'Int': -0.05,
    'Rat': 0.1

}

# Calculate weighted score
te_rec['Score'] = sum(te_rec[col] * weight for col, weight in weights_te.items())

# Sort by score in descending order
ranked_te = te_rec.sort_values(by='Score', ascending=False)

# Get top 10 WR
top_10_te = ranked_te.head(10)
top_10_te[cols].head()

/tmp/ipykernel_18/2090748309.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_rec['Score'] = sum(te_rec[col] * weight for col, weight in weights_te.items())


,Player,Tgt,Rec,Yds,TD,1D,YBC,YAC,ADOT,BrkTkl,Drop%,Drop,Int,Rat
27,Dallas Goedert,20,17,239,0,7.0,81,158,5.2,0,0.0,0,0,116.5
21,Brock Bowers,21,18,197,0,10.0,116,81,5.6,0,0.0,0,0,105.8
50,Isaiah Likely,16,12,141,1,7.0,45,96,7.9,0,0.0,0,0,122.1
36,Mike Gesicki,18,14,156,0,7.0,82,74,7.8,0,5.6,1,0,102.8
38,Hunter Henry,18,12,136,0,9.0,67,69,5.6,0,5.6,1,0,89.1


In [8]:
# List of columns to normalize
cols = [
    'Player', 'Tgt', 'Rec', 'Yds', 'TD', '1D', 'YBC',
    'YAC', 'YAC/R', 'ADOT', 'BrkTkl', 'Drop%', 'Drop',
    'Int', 'Rat'
]


weights_rb = {
    'Tgt': 0.2,
    'Rec': 0.25,
    'Yds': 0.2,
    'TD': 0.15,
    '1D': 0.1,
    'YBC': 0.05,
    'YAC': 0.1,
    'ADOT': 0.05,
    'BrkTkl': 0.1,
    'Drop%': -0.25,
    'Int': -0.05,
    'Rat': 0.01
}

# Calculate weighted score
rb_rec['Score'] = sum(rb_rec[col] * weight for col, weight in weights_rb.items())

# Sort by score in descending order
ranked_rb = rb_rec.sort_values(by='Score', ascending=False)

# Get top 10 WR
top_10_rb = ranked_rb.head(10)
top_10_rb[cols].head()

/tmp/ipykernel_18/1031974494.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rb_rec['Score'] = sum(rb_rec[col] * weight for col, weight in weights_rb.items())


,Player,Tgt,Rec,Yds,TD,1D,YBC,YAC,YAC/R,ADOT,BrkTkl,Drop%,Drop,Int,Rat
31,De'Von Achane,19,17,173,1,8.0,-10,183,10.8,-0.6,1,0.0,0,0,122.1
99,Alvin Kamara,11,10,132,1,2.0,4,128,12.8,0.3,1,9.1,1,0,147.0
32,Breece Hall,19,16,120,1,4.0,19,101,6.3,1.3,1,5.3,1,0,110.5
122,Austin Ekeler,9,9,121,0,6.0,-15,136,15.1,-1.7,2,0.0,0,0,118.7
75,Rachaad White,13,12,98,0,2.0,-45,143,11.9,-3.9,2,0.0,0,0,98.1


In [9]:
passing = pd.read_csv('/kaggle/input/passers-week-3/nfl passers 2024 - Accuracy.csv')
print(passing.head())
print(passing.columns)

   Rk            Player   Tm  Age Pos  G  GS  Cmp  Att  Yds  Bats  ThAwy  \
0   1      Dak Prescott  DAL   31  QB  3   3   74  122  851     3      1   
1   2       Brock Purdy  SFO   25  QB  3   3   69   95  842     0      0   
2   3        Geno Smith  SEA   34  QB  3   3   77  103  787     0      0   
3   4       Jalen Hurts  PHI   26  QB  3   3   72  102  772     2      5   
4   5  Matthew Stafford  LAR   36  QB  3   3   69  101  754     0      4   

   Spikes  Drops  Drop%  BadTh    Bad%  OnTgt  OnTgt%  
0       1      3  2.50%     22  18.30%     83  69.20%  
1       0      4  4.20%     12  12.60%     79  83.20%  
2       1      4  3.90%      9   8.80%     55  53.90%  
3       1      1  1.00%      6   6.30%     49  51.00%  
4       0      6  6.20%     14  14.40%     78  80.40%  
Index(['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'Bats', 'ThAwy', 'Spikes', 'Drops', 'Drop%', 'BadTh', 'Bad%', 'OnTgt',
       'OnTgt%'],
      dtype='object')


In [10]:
passing['TrueCmp%'] = passing['Cmp'] / (passing['Att'] - passing['Spikes'])
passing['Score'] = (passing['TrueCmp%'] * 3) + (passing['Yds'] * 1.25) + (passing['Drops'] * 2) + (passing['BadTh'] * -3) + (passing['OnTgt'] * 3)
ranked_qb = passing.sort_values(by='Score', ascending=False)
col = ['Player', 'Tm', 'Att', 'TrueCmp%', 'Yds', 'Drops', 'BadTh', 'OnTgt%']
ranked_qb[col].head(5)

,Player,Tm,Att,TrueCmp%,Yds,Drops,BadTh,OnTgt%
1,Brock Purdy,SFO,95,0.726316,842,4,12,83.20%
0,Dak Prescott,DAL,122,0.611570,851,3,22,69.20%
4,Matthew Stafford,LAR,101,0.683168,754,6,14,80.40%
2,Geno Smith,SEA,103,0.754902,787,4,9,53.90%
3,Jalen Hurts,PHI,102,0.712871,772,1,6,51.00%
